<a href="https://colab.research.google.com/github/anuj0456/machine_learning_examples/blob/master/EffecientB7_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from datetime import datetime 
import pytz 
IST = pytz.timezone('Asia/Kolkata') 
datetime = datetime.now(IST) 
print("Last run at:", datetime.strftime('%d-%m-%Y at %H:%M:%S %p')) 

Last run at: 13-01-2021 at 12:08:17 PM


In [41]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import layers
import os
import csv
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import sys
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
print(sys.version)
print(tf.__version__)

3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
2.4.0


In [42]:
os.chdir('/content/drive/MyDrive/ML/holiday_season/')
if not os.path.exists('train'):
  !unzip test.zip
  !unzip train.zip

In [43]:
df = pd.read_csv('train.csv')
pd.set_option('max_colwidth', 800)
df['Class'] = df['Class'].astype(str)
df['path'] = df['Image'].map(lambda x:'train/'+x)
df = df.drop(columns=['Image'])
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

In [44]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df['Class'] = df['Class'].astype(str)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'Airplane': 0, 'Candle': 1, 'Christmas_Tree': 2, 'Jacket': 3, 'Miscellaneous': 4, 'Snowman': 5}


In [45]:
df.head(10)

,Class,path
0,4,train/image8250.jpg
1,2,train/image1520.jpg
2,2,train/image9840.jpg
3,1,train/image3391.jpg
4,4,train/image275.jpg
5,3,train/image39.jpg
6,4,train/image2693.jpg
7,0,train/image1321.jpg
8,2,train/image1619.jpg
9,5,train/image431.jpg


In [46]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1)

In [47]:
TARGET_WIDTH = 80
TARGET_HEIGHT = 60

In [48]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255,
                                rotation_range=90,
                                zoom_range=[0.5,1.0],
                                horizontal_flip=True,
                                height_shift_range=0.5,
                                brightness_range=[0.2,1.0],                
                                width_shift_range=[-200,200],
                                vertical_flip=True,
                               ).flow_from_dataframe(
                                train,
                                directory=None,
                                x_col="path",
                                y_col="Class",
                                target_size=(TARGET_WIDTH, TARGET_HEIGHT),
                                color_mode="rgb",
                                classes=None,
                                class_mode="categorical",
                                batch_size=64,
                                shuffle=True,
                                )
valid_gen = ImageDataGenerator(rescale=1./255,).flow_from_dataframe(
                                test,
                                directory=None,
                                x_col="path",
                                y_col="Class",
                                target_size=(TARGET_WIDTH, TARGET_HEIGHT),
                                color_mode="rgb",
                                classes=None,
                                class_mode="categorical",
                                batch_size=64,
                                shuffle=True,
                            )

Found 5822 validated image filenames belonging to 6 classes.
Found 647 validated image filenames belonging to 6 classes.


In [49]:
from tensorflow.keras.applications import EfficientNetB7

def get_model():
    core = EfficientNetB7(
                        include_top=False, input_tensor=None,weights='imagenet',input_shape=(TARGET_WIDTH, TARGET_HEIGHT,3))
    model = core.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dropout(0.2)(model)
    model = layers.Dense(6, activation = "softmax")(model)
    model = tf.keras.Model(inputs=[core.input], outputs=[model])
    
    return model


In [50]:
model = get_model()
model.compile(optimizer=keras.optimizers.Adam(lr = 0.1),metrics=['accuracy'], loss='categorical_crossentropy')

In [51]:
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.1, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [ ]:
history = model.fit(train_gen,
                              epochs=50,
                              verbose=1,
                              validation_data=valid_gen,
                              callbacks = [early_stop, reduce_lr],)

Epoch 1/50
29/91 [========>.....................] - ETA: 21:36 - loss: 18.0441 - accuracy: 0.2199

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
try:
  os.remove('submission.csv')
except Exception as e:
  print("No file exists")
with open('submission.csv', 'w', newline='') as file:
      writer = csv.writer(file, delimiter = ',')
      writer.writerow(["Image", "Class"])

In [ ]:
for filename in os.listdir("test"):
    image = Image.open('test/'+filename)
    image = image.resize((TARGET_HEIGHT, TARGET_WIDTH)).convert("RGB")
    # image = ImageOps.grayscale(image) 
    image = np.expand_dims(image, axis = 0)
    try:
      pred = np.argmax(model.predict(image))
    except Exception as e:
      print(e)
      print(filename)
    labels=['Airplane', 'Candle', 'Christmas_Tree', 'Jacket', 'Miscellaneous','Snowman']
    with open('submission.csv', 'a') as file:
      writer = csv.writer(file, delimiter = ',')
      writer.writerow([filename, labels[pred]])

In [ ]:
from google.colab import files
files.download('submission.csv')